# Implementação didática do RSA com Python puro

In [1]:
# Etapa 1: Escolher dois primos pequenos
p = 23 # 54654656477
q = 17 # 34654655543
n = p * q
print("n =", n)

n = 391


In [2]:
# Etapa 2: Calcular a função totiente de Euler φ(n)
phi = (p - 1) * (q - 1)
print("phi(n) =", phi)

phi(n) = 352


In [3]:
# Etapa 3: Escolher e tal que 1 < e < phi e gcd(e, phi) = 1

def mdc(a, b):
    while b != 0:
        a, b = b, a % b
    return a

e = 65537  # Tentativa
print("gcd(e, phi):", mdc(e, phi))

gcd(e, phi): 1


In [4]:
# Etapa 4: Calcular o inverso modular d de e mod phi usando o algoritmo de Euclides estendido

def egcd(a, b):
    if b == 0:
        return (1, 0)
    else:
        x, y = egcd(b, a % b)
        return (y, x - (a // b) * y)

def modinv(e, phi):
    x, _ = egcd(e, phi)
    return x % phi

d = modinv(e, phi)
print("d =", d)

d = 65


In [5]:
# Etapa 5: Gerar as chaves pública e privada
public_key = (e, n)
private_key = (d, n)
print("Chave pública:", public_key)
print("Chave privada:", private_key)

Chave pública: (65537, 391)
Chave privada: (65, 391)


In [17]:
# Etapa 6: Cifrar uma mensagem M com a chave pública
def message_to_int(m):
   mb= m.encode('utf-8')
   mi= int.from_bytes(mb,byteorder='big')
   return mi
M =  message_to_int("S") # Mensagem original (deve ser < n)
assert M < n
print("M em int:", M)
C = pow(M, e, n)
print("Mensagem criptografada:", C)

M em int: 83
Mensagem criptografada: 304


In [18]:
# Etapa 7: Decifrar com a chave privada
def int_to_message(m):
    return m.to_bytes((m.bit_length() + 7) // 8, byteorder='big').decode('utf-8')
decrypted = pow(C, d, n)
print("Mensagem decriptada:", decrypted, "->",int_to_message(decrypted))
# Verificação final
assert M == decrypted
print("✅ Mensagem decifrada corretamente!")

Mensagem decriptada: 83 -> S
✅ Mensagem decifrada corretamente!


In [8]:
# 🔓 Tentativa de quebra 1: fatorar n para encontrar p e q

def fatorar_n(n):
    for i in range(2, n):
        if n % i == 0:
            return i, n // i
    return None, None

fp, fq = fatorar_n(n)
print("Fatores encontrados:", fp, fq)
print("n=",fp*fq)

Fatores encontrados: 17 23
n= 391


In [52]:
# 🔓 Tentativa de quebra 2: logaritmo discreto (força bruta)

def log_discreto(C, e, n):
    for m in range(n):
        if pow(m, e, n) == C:
            return m
    return None

brute_force_message = log_discreto(C, e, n)
print("Mensagem recuperada via força bruta:", brute_force_message)

Mensagem recuperada via força bruta: 83
